In [5]:
!pip install pyngrok flask requests ffmpeg-python tqdm opencv-python moviepy

In [6]:
!ngrok config add-authtoken 2rJRJd3ZKxaKwd88acgbWZvJKkc_2hicri7WRLoLwBw9KmA47

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
pip install torch==1.13.1 torchvision==0.14.1


  Using cached torch-1.13.1-cp311-cp311-manylinux1_x86_64.whl.metadata (24 kB)
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.14.1 (from versions: 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1)
ERROR: No matching distribution found for torchvision==0.14.1


In [3]:
pip install urllib3 --upgrade

In [4]:
!pip show torch torchvision


Name: torch
Version: 2.5.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchvision
---
Name: torchvision
Version: 0.20.1+cu121
Summary: image and video datasets and models for torch deep learning
Home-page: https://github.com/pytorch/vision
Author: PyTorch Core Team
Author-email: soumith@pytorch.org
License: BSD
Location: /usr/local/lib/python3.11/dist-packages
Requires: numpy, pillow, torch
Required

In [9]:
# wav2lip_colab_server.py
# -----------------------------------------
# 1) Installs pyngrok & other libs
# 2) Clones & sets up your EXACT Wav2Lip-GFPGAN pipeline
# 3) Fixes the requirement "torchvision>=>=0.8.2" using sed
# 4) Runs a Flask server with pyngrok on port 5000
# 5) Accepts POST /generate-lipsync => returns final video

!pip install pyngrok flask requests ffmpeg-python tqdm opencv-python moviepy

import os
import cv2
import ffmpeg
import requests
import shutil
import moviepy.editor as mpe
from tqdm import tqdm
from flask import Flask, request, send_file, jsonify
from pyngrok import ngrok

################################################################################
# (A) Clone Wav2Lip-GFPGAN, fix requirements, download weights
################################################################################
!git clone https://github.com/ajay-sainy/Wav2Lip-GFPGAN.git
basePath = "/content/Wav2Lip-GFPGAN"

wav2lipFolderName = "Wav2Lip-master"
gfpganFolderName  = "GFPGAN-master"

wav2lipPath = os.path.join(basePath, wav2lipFolderName)
gfpganPath  = os.path.join(basePath, gfpganFolderName)

# Download s3fd.pth
!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" \
     -O "{wav2lipPath}/face_detection/detection/sfd/s3fd.pth"

# Download checkpoints (wav2lip.pth)
!gdown https://drive.google.com/uc?id=1fQtBSYEyuai9MjBOF8j7zZ4oQ9W2N64q \
       --output "{wav2lipPath}/checkpoints/wav2lip.pth"

# Move into Wav2Lip-GFPGAN folder
%cd "{basePath}"

# --- FIX the "torchvision>=>=0.8.2" line if present ---
!sed -i 's/torchvision>=>=0.8.2/torchvision>=0.8.2/' requirements.txt

# Install the patched requirements
!pip install -r requirements.txt

# Create inputs/outputs folders if missing
inputsPath  = os.path.join(basePath, "inputs")
outputsPath = os.path.join(basePath, "outputs")
os.makedirs(inputsPath, exist_ok=True)
os.makedirs(outputsPath, exist_ok=True)

################################################################################
# (B) Flask server with pyngrok
################################################################################
app = Flask(__name__)

@app.route("/")
def index():
    return "Colab Wav2Lip + GFPGAN server is running with pyngrok!"

@app.route("/generate-lipsync", methods=["POST"])
def generate_lipsync():
    """
    1) Receives:
       - audio (MP3)
       - video (MP4)
       - subtitles (SRT) [optional]
    2) Runs your EXACT Wav2Lip pipeline for lip-syncing with GFPGAN enhancements.
    3) Merges audio at the end, then optionally merges subtitles.
    4) Returns final MP4 to the client.
    """
    try:
        # 1) Save incoming files
        audio_file = request.files.get("audio")
        video_file = request.files.get("video")
        srt_file   = request.files.get("subtitles")

        if not audio_file or not video_file:
            return jsonify({"error": "Missing audio or video file"}), 400

        # Clear old inputs/outputs
        if os.path.exists(inputsPath):
            shutil.rmtree(inputsPath)
        if os.path.exists(outputsPath):
            shutil.rmtree(outputsPath)
        os.makedirs(inputsPath, exist_ok=True)
        os.makedirs(outputsPath, exist_ok=True)

        inputAudioPath = os.path.join(inputsPath, "output.mp3")
        inputVideoPath = os.path.join(inputsPath, "video.mp4")

        with open(inputAudioPath, "wb") as f:
            f.write(audio_file.read())
        with open(inputVideoPath, "wb") as f:
            f.write(video_file.read())

        srtPath = None
        if srt_file:
            # Put subtitles in "subtitles" folder
            subsDir = os.path.join(basePath, "subtitles")
            if os.path.exists(subsDir):
                shutil.rmtree(subsDir)
            os.makedirs(subsDir, exist_ok=True)
            srtPath = os.path.join(subsDir, "transcript.srt")
            with open(srtPath, "wb") as f:
                f.write(srt_file.read())

        # 2) Wav2Lip inference
        # cd into Wav2Lip folder, run inference.py
        lipSyncedOutputPath = os.path.join(outputsPath, "result.mp4")

        %cd "{wav2lipPath}"
        !python inference.py \
            --checkpoint_path checkpoints/wav2lip.pth \
            --face "{inputVideoPath}" \
            --audio "{inputAudioPath}" \
            --outfile "{lipSyncedOutputPath}"

        # 3) GFPGAN setup & run
        %cd "{gfpganPath}"
        !python setup.py develop
        !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth \
            -P experiments/pretrained_models

        # We'll restore frames from `result.mp4`
        import cv2
        import numpy as np
        from os import path

        inputVideoPath2 = lipSyncedOutputPath
        unProcessedFramesFolderPath = os.path.join(outputsPath, "frames")
        os.makedirs(unProcessedFramesFolderPath, exist_ok=True)

        vidcap = cv2.VideoCapture(inputVideoPath2)
        numberOfFrames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        print("FPS:", fps, "Frames:", numberOfFrames)

        for frameNum in tqdm(range(numberOfFrames)):
            success, image = vidcap.read()
            if not success:
                break
            cv2.imwrite(
                path.join(unProcessedFramesFolderPath, f"{frameNum:04d}.jpg"),
                image
            )

        # Possibly upgrade torchvision if needed
        !pip install --upgrade torchvision

        # GFPGAN inference
        !python inference_gfpgan.py \
          -i "{unProcessedFramesFolderPath}" \
          -o "{outputsPath}" \
          -v 1.3 -s 2 --only_center_face --bg_upsampler None

        # Reassemble frames in batches of 300
        restoredFramesPath = os.path.join(outputsPath, "restored_imgs")
        dir_list = sorted(os.listdir(restoredFramesPath))
        batch = 0
        batchSize = 300

        for i in tqdm(range(0, len(dir_list), batchSize)):
            img_array = []
            start, end = i, i + batchSize
            print("processing", start, end)

            for filename in dir_list[start:end]:
                filepath = os.path.join(restoredFramesPath, filename)
                img = cv2.imread(filepath)
                if img is None:
                    continue
                height, width, layers = img.shape
                size = (width, height)
                img_array.append(img)

            outPath = os.path.join(outputsPath, f"batch_{batch:04d}.avi")
            out = cv2.VideoWriter(outPath, cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
            batch += 1

            for frame in img_array:
                out.write(frame)
            out.release()

        # concat the batch AVIs
        concatTextFilePath = os.path.join(outputsPath, "concat.txt")
        with open(concatTextFilePath, "w") as ctf:
            for idx in range(batch):
                ctf.write(f"file batch_{idx:04d}.avi\n")

        concatedVideoOutputPath = os.path.join(outputsPath, "concated_output.avi")
        !ffmpeg -y -f concat -i "{concatTextFilePath}" -c copy "{concatedVideoOutputPath}"

        finalProcessedOutputVideo = os.path.join(outputsPath, "final_with_audio.avi")
        !ffmpeg -y -i "{concatedVideoOutputPath}" -i "{inputAudioPath}" \
                 -map 0 -map 1:a -c:v copy -shortest "{finalProcessedOutputVideo}"

        # 4) Merge subtitles if provided
        final_no_subs = os.path.join(outputsPath, "final_no_subs.mp4")
        !ffmpeg -y -i "{finalProcessedOutputVideo}" -c copy "{final_no_subs}"

        final_with_subs = os.path.join(outputsPath, "final_with_subs.mp4")
        if srtPath and os.path.exists(srtPath):
            (
                ffmpeg
                .input(final_no_subs)
                .output(final_with_subs, vf=f"subtitles={srtPath}", strict='experimental')
                .run(overwrite_output=True)
            )
            outputToReturn = final_with_subs
        else:
            outputToReturn = final_no_subs

        print(f"Returning file: {outputToReturn}")
        return send_file(outputToReturn, as_attachment=True)

    except Exception as e:
        return jsonify({"error": str(e)}), 500

# (C) Create the ngrok tunnel on port 5000
public_url = ngrok.connect(5000)
print("Ngrok Tunnel URL:", public_url)

# (D) Run the Flask server
app.run(host='0.0.0.0', port=5000)


  if event.key is 'enter':



Cloning into 'Wav2Lip-GFPGAN'...
remote: Enumerating objects: 195, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 195 (delta 18), reused 9 (delta 9), pack-reused 160 (from 1)
Receiving objects: 100% (195/195), 29.94 MiB | 40.71 MiB/s, done.
Resolving deltas: 100% (39/39), done.
--2025-01-19 02:30:27--  https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth
Resolving www.adrianbulat.com (www.adrianbulat.com)... 45.136.29.207
Connecting to www.adrianbulat.com (www.adrianbulat.com)|45.136.29.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89843225 (86M) [application/octet-stream]
Saving to: ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’

/content/Wav2Lip-GF 100%[===================>]  85.68M  20.6MB/s    in 4.9s    

2025-01-19 02:30:33 (17.3 MB/s) - ‘/content/Wav2Lip-GFPGAN/Wav2Lip-master/face_detection/detection/sfd/s3fd.pth’ saved [89843225/898

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
